# Pandas'ı Doğru Kullanarak Kodumuzu Hızlandırma
### Bu çalışmada Pandas'ı doğru kullanarak kodumuzu nasıl hızlandırabileceğimizi göreceğiz.
Bu notebook size Pandas kütüphanesinin nasıl tasarlandığı şekilde kullanılacağını ve matris işlemleri açısından nasıl düşünüleceğini öğretecektir. Bu arada, size Pandas'ın kodunuzdaki for döngülerinden daha hızlı çalışmasını sağlayacak, pratik zaman kazandıran birkaç ipucu ve püf noktası göstereceğim!

Bu kod boyunca, klasik Iris Flowers veri setini kullanacağız. Veri setini seaborn ile yükleyerek ve ilk 5 satırı yazdırarak başlayalım.

In [1]:
import seaborn as sns
import pandas as pd

data = sns.load_dataset('iris')

print(data.head())

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa


Şimdi hızımızı Python for-loop ile ölçmek için bir temel oluşturalım. Veri setimizde gerçekleştirilecek bir hesaplama oluşturacağız, her satırdan geçerek tüm işlemin hızını ölçeceğiz. Bu, yeni optimizasyonlarımızın bize ne kadar yardımcı olduğunu görmek için bize bir temel sağlayacaktır.

In [2]:
import seaborn as sns
import time

data = sns.load_dataset('iris')

def compute_class(petal_length):
    if petal_length <= 2:
        return 1
    elif 2 < petal_length < 5:
        return 2
    else:
        return 3

start = time.time()

class_list = list()
for i in range(len(data)):
    petal_length = data.iloc[i]['petal_length']
    class_num = compute_class(petal_length)
    class_list.append(class_num)

end = time.time()
print("For-loop run time = {}".format(end - start))

For-loop run time = 0.031827688217163086


Yukarıdaki kodda, bir If-Else deyimi kullanarak taç uzunluğuna göre çiçeğin sınıfını seçen temel bir fonksiyon yarattık. Dataframe içinde loop yaparak her satırdaki fonksiyonları uygulayan bir for-loop yazdık ve daha sonra loop'un toplam çalışma süresini ölçtük.

Colab'da döngü, 5 çalıştırmada ortalama 0.031827 saniye sürdü.

# .İterrows () ile döngü
Yapabileceğimiz en kolay ancak çok değerli hız Panda'nın yerleşik  .iterrows()işlevini kullanmaktır.

For-loop'umuzu önceki bölümde yazdığımızda, range()fonksiyonu kullanıyorduk . Yine de Python'da geniş bir değer aralığında döngü oluştururken, jeneratörler çok daha hızlı olma eğilimindedir. 

Aşağıda, .iterrows() düzenli bir for döngüsü yerine kullanılacak kodu değiştirdik  . Colab'da, ortalama çalışma süresi 0.018173933029174805 saniye idi - bir 1.75X hızlanma!

In [6]:
0.031827688217163086/0.018173933029174805

1.751282354021541

In [3]:
import seaborn as sns
import time

data = sns.load_dataset('iris')

def compute_class(petal_length):
    if petal_length <= 2:
        return 1
    elif 2 < petal_length < 5:
        return 2
    else:
        return 3

start = time.time()

class_list = list()
for index, data_row in data.iterrows():
    petal_length = data_row['petal_length']
    class_num = compute_class(petal_length)
    class_list.append(class_num)

end = time.time()
print("Iterrows run time = {}".format(end - start))

Iterrows run time = 0.018173933029174805


# .apply ile for döngüsünden kurtulma

In [4]:
import seaborn as sns
import time

data = sns.load_dataset('iris')

def compute_class(petal_length):
    if petal_length <= 2:
        return 1
    elif 2 < petal_length < 5:
        return 2
    else:
        return 3

start = time.time()

class_list = data.apply(lambda row: compute_class(row['petal_length']), axis=1)

end = time.time()
print(".apply() run time = {}".format(end - start))

.apply() run time = 0.007298946380615234


Süre daha da kısa sürdü: For döngüsüne göre X4,36 hız!!

In [7]:
0.031827688217163086/0.007298946380615234

4.3605866596981775

# .cut() metodu ile hızı daha da artırma

In [5]:
import seaborn as sns
import time
import pandas as pd

data = sns.load_dataset('iris')

start = time.time()

class_list = pd.cut(x=data.petal_length,
                   bins=[0, 2, 5, 100],
                   include_lowest=True,
                   labels=[1, 2, 3]).astype(int)

end = time.time()
print(".cut() run time = {}".format(end - start))

.cut() run time = 0.0057163238525390625


Son yöntemle hızımız 5 kat arttı

In [8]:
0.031827688217163086/0.0057163238525390625

5.567859526192859

### Faydalı olması dileğiyle